In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = np.array(json.loads(open(data_path, "r").read())['col_name'])

create_dummy(data_path, dummy_data_path)
dummy_data = np.array(json.loads(open(dummy_data_path, "r").read())['col_name'])

dummy_theory_output = torch.tensor(mode_within(dummy_data[dummy_data>20],1))

In [6]:
scales = [2]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    filter = (x > 20)
    filtered_x = s.where(filter, x)
    return s.mode(filtered_x)

error = 0.01
_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:228: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = torch.tensor(mode_within(x_1d, 0))
/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this functi

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 2, param_scale: 2, scale_input_multiplier: 10) ------------->

+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.024999619 | 0.049999237  | 0.049999237 | 0         | 0.024999619    | 0.049999237      | 0.049999237   | 0             | 0.0012499619       | 0.0005580272       | 0.0005580272           |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---

scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-636,640],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":12534,"total_assignments":25069,"total_const_size":8414,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[3936,[1]]},"required_lookups":["ReLU",{"GreaterThan":{"a":0.0}},{"Cast":{"scale":4.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715324338361}


In [8]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.7117280960083008 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 44.75
==== Generating Proof ====
proof:  {'instances': [['f660de1c93e554bceb59df902e1abc5c5e332e2541da829619b3974ff2efbe21', '0100000000000000000000000000000000000000000000000000000000000000', 'b300000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x03b2c98a15ccde53e41aa8223dbd7dc9a767e5d8d0ec47a56b1c2a777250535f027836ecd2c5e6de44186b357b1c5bcb9f82a028ceff7df644bea0386119b76903a6627df30eb420a6dd2a6d83669cb79d2a961177aff1c01b8ef983aa87c291107439d5ff968c91bd09c39b9d6b877aa0be18d4297e6ee37dba1ee64708af5022b1142f943e9b96f8db070271297aa65f3647f4bb6237179b4637b2af4ebd1a1989c4ef7e1af7682e44931215a00924ada5fcd1f014f9d4d7d0eae9cea10e0008dedb5158c5c59be6c1540e9dc81162cc340088f8c749196976235b299e57400eeafcb99d42a5b6aa8228d5eed21bc2b3e10a2c7d61fe7cfe3f9c21f50ecacf2e337fd4443961d1e4665dc0ca5fd3619f0367ac7f0ef33443dd7bcbb0f1466510162dc7c3d59f

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [44.75]
